In [39]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np
import pickle
import lftk
import spacy
nlp = spacy.load("en_core_web_sm")

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('/data/pbcalder/home77/Documents/hierarchical-hawkes/on-VEVO/reduced_textvecs/my-32dim-model')

import pickle
import os
import pandas as pd

In [1]:
from numpy.random import multivariate_normal
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import os
import pandas as pd

# get vecs

## theta

In [40]:
#https://gist.github.com/bstonedev/9bc7f73001e24c5a73931f00955e0b1f
fixer = {"‚Äô": "'",
"‚Äô": "'",
"‚Äò": "'",
"‚Äã": "",
"‚Äã": "",
"‚Ä¶": "...",
"‚Ä¢": "-",
"‚Äê": "-",
"-¬≠": "-",
"‚Äú": "'",
"‚Äù": "'",
"&nbsp;": " ",
"¬†": " ",
"‚Äì": "–",
"¬Æ": "®",
"&amp;": "&",
"‚Äö√Ñ√¨": "-"}
def fix(st):
    for word, initial in fixer.items():
        st = st.replace(word, initial)
    return st

si, ss, pca = pickle.load(open("/data/pbcalder/home77/Documents/hierarchical-hawkes/rnixcnix/lftk/imputer_scaler_pca.p", "rb"))

families = [
    "wordsent",
"partofspeech",
"entity",
"avgwordsent",
"avgpartofspeech",
"avgentity",
"lexicalvariation",
"typetokenratio",
"readformula",
"readtimeformula"]

search_feats = []
for family in families:
    search_feats.extend(lftk.search_features(family = family, return_format = "list_key"))
    
def get_lftk_text_vector(t):
    try:
        doc = nlp(t)
        LFTK = lftk.Extractor(docs = doc)
        LFTK.customize(stop_words=True, punctuations=False, round_decimal=3)
        extracted_features = LFTK.extract(features=search_feats)#features = selected)
        return np.array(list(extracted_features.values()))
    except:
        return np.full([211], np.nan)

def apply_style_pipe(t):
    return pca.transform(ss.transform(si.transform(get_lftk_text_vector(t).reshape(1,-1))))

/home/pbcalder/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator SimpleImputer from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/pbcalder/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/pbcalder/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator PCA from version 0.24.2 when using version 1.0.2. This might lead to breaking code or

In [42]:
relevant_df = pd.read_csv('whatif-news.com.au - Sheet1.csv')

In [ ]:
array(["'She was just 13': Albanese's emotional tribute to Bali bombing victims",
       "Foreign policy and the Albanese government's first 100 days",
       'Stamp duty reform should be allowed to go ahead',
       'Stars collide in battle for Jillaroos No.1 jersey at the World Cup',
       "Cryptocurrency company accidentally transfers $10.5m to Australian woman and doesn't notice for seven months"],


In [43]:
big_df = pd.read_csv("../../../inflammatory/concated.csv")
body_dict, relevant_index = pickle.load(open("../../../inflammatory/theta/relevant_bigdf_index.p", "rb"))
big_df = big_df.loc[relevant_index]

In [45]:
big_df['Body'] = big_df['Article id'].map(body_dict)

In [46]:
big_df = pd.concat([
    big_df[big_df["Headline"].map(lambda x: 'She was just 13' in x)],
    big_df[big_df["Headline"].map(lambda x: 'Foreign policy and the Albanese government' in x)],
    big_df[big_df["Headline"].map(lambda x: 'Stamp duty reform should be allowed' in x)],
    big_df[big_df["Headline"].map(lambda x: 'Stars collide in battle' in x)],
    big_df[big_df["Headline"].map(lambda x: 'Cryptocurrency company accidentally transfers' in x)]
], axis=0)
    

In [57]:
bodies = [big_df["Body"].values[0]] * 51 + \
[big_df["Body"].values[1]] * 51 + \
[big_df["Body"].values[2]] * 51 + \
[big_df["Body"].values[3]] * 51 + \
[big_df["Body"].values[4]] * 51

In [65]:
relevant_df = relevant_df.rename({"headline": "Headline"}, axis=1)

In [66]:
relevant_df["Body"] = bodies

In [67]:
vecs_tb = np.hstack([
    model.encode(relevant_df["Headline"].values), # title semantic
    np.stack(relevant_df["Body"].map(lambda x: apply_style_pipe(x)[0]).values), # body style
])

In [68]:
base_data = pickle.load(open(
    "/data/pbcalder/rnixcnix-bayes/stan/THETA_0x(m)_0y(t+b)_ATNIX_minE10_maxE50_maxC100_full.p", "rb"))

/home/pbcalder/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [69]:
dats = []
for publisher_index in [16]:#range(28):
    logo_data = base_data[int(publisher_index)]
    publisher_name = logo_data[0]
    publisher_y_scaler = logo_data[1][2]
    scaled_vecs = publisher_y_scaler.transform(np.stack(vecs_tb))
    dats.append([publisher_index, publisher_name, scaled_vecs])
    print(publisher_index)
pickle.dump(dats, open("scaled_vecs_theta_16.p", "wb"))

16


## alpha

In [70]:
logo_data = pickle.load(open("/data/pbcalder/rnixcnix-bayes/stan/LOGO_0x(m)_0y(t)_ATNIX_full.p", "rb"))

/home/pbcalder/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [73]:
textvecs = model.encode(relevant_df["Headline"].values)

In [74]:
dats = []
for publisher_idx in [16]:#range(len(logo_data)):
    publisher_name = logo_data[publisher_idx][0]
    publisher_y_scaler = logo_data[publisher_idx][1][2]
    scaled_vecs = publisher_y_scaler.transform(np.stack(textvecs))
    dats.append([publisher_idx, publisher_name, scaled_vecs])

In [75]:
pickle.dump(dats, open("scaled_vecs_16.p", "wb"))

In [177]:
!pwd

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/pbcalder/Documents/analyse_bayesian/inflammatory_rnix/theta/single_analysis


# run model

## theta

In [78]:
# number of fitted models from FAKENIX
model_indices = [16] # news.com.au

In [79]:
def softmax(a, axis=None):
    """
    Computes exp(a)/sumexp(a); relies on scipy logsumexp implementation.
    :param a: ndarray/tensor
    :param axis: axis to sum over; default (None) sums over everything
    """
    from scipy.special import logsumexp
    lse = logsumexp(a, axis=axis)  # this reduces along axis
    if axis is not None:
        lse = np.expand_dims(lse, axis)  # restore that axis for subtraction
    return np.exp(a - lse)


def get_hl(log_thet, log_cc):
    return log_cc + np.log((2 ** (1 / np.exp(log_thet)) - 1))

def get_halftime(x):
    if len(x) % 2 == 1:
        return (x[(len(x) - 1) // 2])/60/60
    else:
        return (0.5 * (x[len(x) // 2] + x[(len(x) // 2) - 1]))/60/60

def get_average_halftime(l):
    return np.median([get_halftime(x) for x in l])

def get_halftimes(l):
    return [get_halftime(x) for x in l]

In [81]:
dats = pickle.load(open("scaled_vecs_theta_16.p", "rb"))

In [84]:
pop_pvals = []
unpop_pvals = []
z_pvals = []
expected_pvals = []

ys_over_models = []
for model_index in model_indices:
    
    ys = [] # log median halflife
    vecs_all = dats[0][2]
    for selected_test_article_index in range(len(relevant_df)):
        scaled_vecs_for_user = vecs_all[selected_test_article_index, :]
        sdraws = pd.read_csv(f"../standraws_with_cov/{model_index}.csv")

        gamma_a_mid = sdraws[[x for x in sdraws.columns if 'gamma_a_plmid[' in x]].values
        gamma_a_fast = sdraws[[x for x in sdraws.columns if 'gamma_a_plfast[' in x]].values
        gamma_a_slow = sdraws[[x for x in sdraws.columns if 'gamma_a_plslow[' in x]].values
        
        beta_fast = sdraws[
            ['beta_a_plfast[1]', 'beta_a_plfast[2]']].values
        
        covariance_fast = sdraws[[
            'covariance_beta_plfast[1,1]',
            'covariance_beta_plfast[2,1]',
            'covariance_beta_plfast[1,2]',
            'covariance_beta_plfast[2,2]'
        ]].values
        
        ###
        
        beta_slow = sdraws[
            ['beta_a_plslow[1]', 'beta_a_plslow[2]']].values
        
        covariance_slow = sdraws[[
            'covariance_beta_plslow[1,1]',
            'covariance_beta_plslow[2,1]',
            'covariance_beta_plslow[1,2]',
            'covariance_beta_plslow[2,2]'
        ]].values
        
        ###
        
        beta_mid = sdraws[
            ['beta_a_plmid[1]', 'beta_a_plmid[2]']].values
        
        covariance_mid = sdraws[[
            'covariance_beta_plmid[1,1]',
            'covariance_beta_plmid[2,1]',
            'covariance_beta_plmid[1,2]',
            'covariance_beta_plmid[2,2]'
        ]].values
        
        ###
        
        beta_z = sdraws[
            ['beta_a_z[1]', 'beta_a_z[2]']].values
        
        covariance_z = sdraws[[
            'covariance_beta_z[1,1]',
            'covariance_beta_z[2,1]',
            'covariance_beta_z[1,2]',
            'covariance_beta_z[2,2]'
        ]].values
        
        vecs = vecs_all[selected_test_article_index]
        
        hls = np.zeros(shape=(500, 3))
        ps = np.zeros(shape=(500, 3))
        for s in range(500):
            log_theta_fast, log_c_fast = multivariate_normal(
            beta_fast[s],
            covariance_fast[s].reshape(2,2))
        
            log_theta_mid, log_c_mid = multivariate_normal(
                beta_mid[s],
                covariance_mid[s].reshape(2,2))
            
            log_theta_slow, log_c_slow = multivariate_normal(
                beta_slow[s],
                covariance_slow[s].reshape(2,2))
            
            log_theta_fast += np.dot(scaled_vecs_for_user, gamma_a_fast[s])
            log_theta_mid += np.dot(scaled_vecs_for_user, gamma_a_mid[s])
            log_theta_slow += np.dot(scaled_vecs_for_user, gamma_a_slow[s])

            hl_fast = (get_hl(log_theta_fast, log_c_fast))
            hl_mid = (get_hl(log_theta_mid, log_c_mid))
            hl_slow = (get_hl(log_theta_slow, log_c_slow))
        
            hls[s,0] = hl_mid 
            hls[s,1] = hl_slow 
            hls[s,2] = hl_fast 

            beta_z_slow, beta_z_fast = multivariate_normal(
                beta_z[s],
                covariance_z[s].reshape(2,2))
                
            p_mid, p_slow, p_fast = softmax([1, beta_z_slow, beta_z_fast])
        
            ps[s,0] = p_mid 
            ps[s,1] = p_slow 
            ps[s,2] = p_fast 

        
        ys.append(np.log(np.median(np.sum(np.exp(hls) * ps, axis=1))))

    ys_over_models.append([model_index, ys, relevant_df['class'].values])

pickle.dump(ys_over_models, open("ys_over_models_theta.p", "wb"))

## alpha

In [97]:
# number of fitted models from FAKENIX
model_indices = [16]

In [98]:
def inv_logit(x):
    return 1 / (1 + np.exp(-x))

In [103]:
dats = pickle.load(open("scaled_vecs_16.p", "rb"))

In [104]:
pop_pvals = []
unpop_pvals = []
z_pvals = []
expected_pvals = []

ys_over_models = []
for model_index in model_indices:
    # try:
    z_median_list = []
    pop_median_list = []
    unpop_median_list = []
    expected_median_list = []

    ys = []
    
    for selected_test_article_index in range(len(relevant_df)):
        vecs_all = dats[0][2]
        sdraws = pd.read_csv(f"../../standraws_with_cov/{model_index}.csv")
        
        betas = sdraws[[x for x in sdraws.columns if x.startswith('beta_a')]][:500].values
        covariance_betas = sdraws[[x for x in sdraws.columns if 'cov' in x]][:500].values
        gammaz = sdraws[[x for x in sdraws.columns if 'gamma_z_alpha' in x]][:500].values
        gammaa = sdraws[[x for x in sdraws.columns if 'gamma_a_alpha' in x]][:500].values
        
        
        
        vecs = vecs_all[selected_test_article_index]
        
        zs = []
        size_pops = []
        size_unpops = []
        size_expecteds = []
        for s in range(500):
            beta_a1, beta_a2, beta_a3 = multivariate_normal(
                    betas[s],
                    covariance_betas[s].reshape(3,3))
        
            z_new = inv_logit(beta_a1 + np.dot(gammaz[s], vecs.T))
            alpha_popular_new = inv_logit(beta_a2 + np.dot(gammaa[s], vecs.T))
            alpha_unpopular_new = inv_logit(beta_a3 + np.dot(gammaa[s], vecs.T))
        
            size_popular_new = 1 / (1 - alpha_popular_new)
            size_unpopular_new = 1 / (1 - alpha_unpopular_new)
            expected_size_new = (z_new * size_popular_new + (1-z_new) * size_unpopular_new)
        
            zs.append(z_new)
            size_pops.append(size_popular_new)
            size_unpops.append(size_unpopular_new)
            size_expecteds.append(expected_size_new)
        z_median = np.median(zs)
        pop_median = np.median(size_pops)
        unpop_median = np.median(size_unpops)
        expected_median = np.median(size_expecteds)
        
        z_median_list.append(z_median)
        pop_median_list.append(pop_median)
        unpop_median_list.append(unpop_median)
        expected_median_list.append(expected_median)

        # ys is a list over articles
        ys.append(np.log(expected_median))

    ys_over_models.append([model_index, ys, relevant_df['class'].values])
    # except:
    #     ys_over_models.append([model_index,np.nan, np.nan])
        
pickle.dump(ys_over_models, open("ys_over_models_alpha.p", "wb"))

# analyse

In [111]:
relevant_df = pd.read_csv('whatif-news.com.au - Sheet1.csv')

In [112]:
relevant_df['hl_pred'] = pickle.load(open("ys_over_models_theta.p", "rb"))[0][1]

In [113]:
relevant_df['pop_pred'] = pickle.load(open("ys_over_models_alpha.p", "rb"))[0][1]

In [173]:
rels = []
for e in range(1,6):
    rel = relevant_df[relevant_df['example'] == e].copy()
    rel['hl_%diff'] = (np.exp(rel['hl_pred']) - np.exp(rel['hl_pred'].iloc[0])) / np.exp(rel['hl_pred'].iloc[0])
    rel['pop_%diff'] = (np.exp(rel['pop_pred']) - np.exp(rel['pop_pred'].iloc[0])) / np.exp(rel['pop_pred'].iloc[0])
    rel['av_%diff'] = 0.5 * (rel['hl_%diff'] + rel['pop_%diff'])
    rels.append(rel)

In [175]:
relevant_df = pd.concat(rels, axis=0)

In [176]:
relevant_df.to_csv("whatif-filled.csv",index=False)

In [135]:
rel['color'] = rel['class'].map(lambda x: "black" if x=="original" else "blue")

/tmp/ipykernel_3071124/1150521571.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rel['color'] = rel['class'].map(lambda x: "black" if x=="original" else "blue")


"Stars collide in battle for Jillaroos No.1 jersey at the World Cup'"

In [155]:
for k in range(1,6):
    rel = relevant_df[relevant_df['example']==k].copy()
    g = sns.jointplot(data=rel, x="pop_pred", y="hl_pred", hue='class')
    g.plot_joint(sns.kdeplot, color="r", zorder=0, levels=6)
    plt.savefig(f"example_{k}.png", bbox_inches='tight')
    plt.close()

/home/pbcalder/anaconda3/lib/python3.9/site-packages/seaborn/distributions.py:1106: UserWarning: Dataset has 0 variance; skipping density estimate. Pass `warn_singular=False` to disable this warning.
  warnings.warn(msg, UserWarning)
/home/pbcalder/anaconda3/lib/python3.9/site-packages/seaborn/distributions.py:1106: UserWarning: Dataset has 0 variance; skipping density estimate. Pass `warn_singular=False` to disable this warning.
  warnings.warn(msg, UserWarning)
/home/pbcalder/anaconda3/lib/python3.9/site-packages/seaborn/distributions.py:1106: UserWarning: Dataset has 0 variance; skipping density estimate. Pass `warn_singular=False` to disable this warning.
  warnings.warn(msg, UserWarning)
/home/pbcalder/anaconda3/lib/python3.9/site-packages/seaborn/distributions.py:1106: UserWarning: Dataset has 0 variance; skipping density estimate. Pass `warn_singular=False` to disable this warning.
  warnings.warn(msg, UserWarning)
/home/pbcalder/anaconda3/lib/python3.9/site-packages/seaborn/dis